In [2]:
import os
import logging
from google.cloud import storage
import vertexai
from vertexai.generative_models import GenerativeModel, Part

In [3]:
import google.auth
from google.cloud import storage

# Check if credentials are correctly set
credentials, project = google.auth.default()
print(f"Project ID: {project}")
print(f"Credentials: {credentials}")


Project ID: None
Credentials: <google.oauth2.credentials.Credentials object at 0x7f274b0b5d10>


In [4]:
# Configure logging
logging.basicConfig(level=logging.INFO)

# Initialize Vertex AI and Google Cloud Storage
project_id = "flowing-blade-430704-q3"
vertexai.init(project=project_id, location="us-central1")

storage_client = storage.Client(project="flowing-blade-430704-q3")
bucket_name = "gta_images_bucket_01"
bucket = storage_client.bucket(bucket_name)

model = GenerativeModel("gemini-1.5-flash-001")

# Paths
image_dir = "images"
caption_dir = "google_captions_vamsi"

# Ensure the caption directory exists
os.makedirs(caption_dir, exist_ok=True)

In [5]:
def upload_to_gcs(local_path, gcs_path):
    try:
        blob = bucket.blob(gcs_path)
        blob.upload_from_filename(local_path)
        logging.info(f"Uploaded {local_path} to gs://{bucket_name}/{gcs_path}")
        return f"gs://{bucket_name}/{gcs_path}"
    except Exception as e:
        logging.error(f"Failed to upload {local_path} to GCS: {e}")
        return None

In [6]:
def generate_caption(image_gcs_uri):
    try:
        response = model.generate_content(
            [
                Part.from_uri(image_gcs_uri, mime_type="image/jpeg"),
                "Describe the image according to traffic data in 2-3 sentences.",
            ],
            safety_settings={
                "categories": [
                    {"category": "HATE_SPEECH", "threshold": "BLOCK_FEW"},
                    {"category": "DANGEROUS_CONTENT", "threshold": "BLOCK_FEW"},
                    {"category": "SEXUALLY_EXPLICIT", "threshold": "BLOCK_FEW"},
                    {"category": "HARASSMENT", "threshold": "BLOCK_FEW"},
                ]
            }
        )
        logging.info(f"Response: {response}")
        return response.text
    except Exception as e:
        logging.error(f"Failed to generate caption for {image_gcs_uri}: {e}")
        return f"Error generating caption: {e}"

In [7]:
import time

# Add a delay between requests to avoid hitting the quota limit
def rate_limited_generate_caption(image_gcs_uri):
    try:
        # Wait for 30 seconds before making a new request
        time.sleep(30)
        return generate_caption(image_gcs_uri)
    except Exception as e:
        logging.error(f"Failed to generate caption for {image_gcs_uri}: {e}")
        return f"Error generating caption: {e}"

In [8]:
# Initialize a counter for processed images
processed_count = 0
start_image = 1
end_image = 8322

# Create a set of image names from '00001.png' to '08322.png'
image_names = {f"{str(i).zfill(5)}.png" for i in range(start_image, end_image + 1)}

# Traverse through the directory and process only the images within the specified range
for root, dirs, files in os.walk(image_dir):
    # Skip hidden directories like .ipynb_checkpoints
    dirs[:] = [d for d in dirs if not d.startswith('.')]
    
    logging.info(f"Scanning directory: {root}")
    for file in files:
        # Skip hidden files
        if file.startswith('.'):
            continue

        if file.lower() in image_names:
            caption_file_name = f"{os.path.splitext(file)[0]}.txt"
            caption_path = os.path.join(caption_dir, caption_file_name)

            # Skip the image if the caption file already exists
            if os.path.exists(caption_path):
                #logging.info(f"Caption already exists for {file}, skipping...")
                continue
            
            image_path = os.path.join(root, file)
            
            # Upload image to GCS
            gcs_path = f"temp_images/{file}"
            image_gcs_uri = upload_to_gcs(image_path, gcs_path)
            if not image_gcs_uri:
                continue

            # Generate caption with rate limiting
            caption = rate_limited_generate_caption(image_gcs_uri)

            # Save caption to file
            try:
                with open(caption_path, 'w') as caption_file:
                    caption_file.write(caption)
                logging.info(f"Saved caption to {caption_path}")
            except Exception as e:
                logging.error(f"Failed to save caption to {caption_path}: {e}")
            
            # Optionally, delete the uploaded image from GCS
            try:
                bucket.blob(gcs_path).delete()
                logging.info(f"Deleted {gcs_path} from GCS")
            except Exception as e:
                logging.error(f"Failed to delete {gcs_path} from GCS: {e}")

            # Increment the counter
            processed_count += 1

        if processed_count >= len(image_names):
            logging.info("All specified images processed, exiting...")
            break
    if processed_count >= len(image_names):
        break

print("Captions generated and saved successfully.")

INFO:root:Scanning directory: images


Captions generated and saved successfully.


In [9]:
import os

# Range of images
start_image = 1
end_image = 8322

# Generate sets of image names and expected caption names
image_names = {f"{str(i).zfill(5)}.png" for i in range(start_image, end_image + 1)}
caption_names = {f"{str(i).zfill(5)}.txt" for i in range(start_image, end_image + 1)}

# Get the set of existing caption files
existing_captions = {file for file in os.listdir(caption_dir) if file.endswith('.txt')}

# Check for missing captions
print(len(image_names))
print(len(caption_names))
missing_captions = caption_names - existing_captions
if not missing_captions:
    print("Captions generated for all images.")
else:
    print(f"Missing captions for the following images ({len(missing_captions)} missing):")
    for missing_caption in missing_captions:
        print(missing_caption.replace('.txt', '.png'))


8322
8322
Captions generated for all images.


In [13]:
import os
import json
import re

# Directory where the captions are stored
caption_dir = "google_captions_vamsi"

# Function to search for specific text pattern in a file
def search_in_file(file_path):
    try:
        with open(file_path, 'r') as file:
            content = file.read()
            # Check if the specific text pattern exists in the content
            if "Error" in content or "error" in content:
                print(f"Pattern found in file: {file_path}")
                return True
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
    return False

# Traverse through the caption directory and search for the specific pattern
pattern_found_files = []

for root, dirs, files in os.walk(caption_dir):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(root, file)
            if search_in_file(file_path):
                pattern_found_files.append(file_path)

# Log the results
if pattern_found_files:
    print("Files containing the specific text pattern:")
    for file_path in pattern_found_files:
        print(file_path)
else:
    print("No files containing the specific text pattern were found.")


Pattern found in file: google_captions_vamsi/.ipynb_checkpoints/01488-checkpoint.txt
Files containing the specific text pattern:
google_captions_vamsi/.ipynb_checkpoints/01488-checkpoint.txt
